In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import tkinter as tk
from tkinter import ttk

In [2]:
# Đọc dữ liệu từ tệp CSV
data = pd.read_csv('dulieucongviecvn.csv')


In [3]:
# Kết hợp các trường vào một cột để tạo ma trận TF-IDF
data['kethop'] = data['roles'] + " " + data['companies'] + " " + data['locations'] + " " + data['salary'].astype(str)

# Chuyển đổi văn bản thành ma trận TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['kethop'])

# Tạo mô hình KNN
model = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(tfidf_matrix)

def recommend_jobs(query):
    query_vec = vectorizer.transform([query])
    distances, indices = model.kneighbors(query_vec)
    return data.iloc[indices[0]]

In [4]:
class JobRecommenderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Đề xuất công việc")

        # Tạo các trường nhập liệu
        self.roles_var = tk.StringVar()
        self.companies_var = tk.StringVar()
        self.locations_var = tk.StringVar()
        self.salary_var = tk.StringVar()

        self.create_widgets()

    def create_widgets(self):
        # Nhãn và các trường nhập liệu
        ttk.Label(self.root, text="Vai trò:").grid(row=0, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.roles_var).grid(row=0, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Công Ty:").grid(row=1, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.companies_var).grid(row=1, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Vị trí:").grid(row=2, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.locations_var).grid(row=2, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Lương:").grid(row=3, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.salary_var).grid(row=3, column=1, sticky=tk.EW, padx=10, pady=5)
        
        # Nút bấm và khung hiển thị kết quả
        ttk.Button(self.root, text="Đề xuất công việc", command=self.recommend_jobs).grid(row=4, column=0, columnspan=2, pady=10)
        self.results = tk.Text(self.root, height=10, width=50)
        self.results.grid(row=5, column=0, columnspan=2, sticky=tk.NSEW, padx=10, pady=5)

        # Cấu hình grid để mở rộng
        self.root.grid_rowconfigure(5, weight=1)
        self.root.grid_columnconfigure(1, weight=1)

    def recommend_jobs(self):
        query = " ".join([self.roles_var.get(), self.companies_var.get(), self.locations_var.get(), self.salary_var.get()])
        recommendations = recommend_jobs(query)
        self.results.delete(1.0, tk.END)
        for i, job in recommendations.iterrows():
            self.results.insert(tk.END, f"{job['roles']} tại công ty {job['companies']} ở địa điểm {job['locations']} với mức lương {job['salary']}\n\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = JobRecommenderApp(root)
    root.mainloop()